## POS tagging using modified Viterbi

<hr />

1. <B> Problem Statment </B>
 * Implement atleast 2 methods to tag unknown words using Viterbi algorith,. 
    
2. <B> DataSet </B>
 *  Penn Treebank dataset of NLTK (with 'universal' tagset)
 *  The dataset consisis of 12 POS tags which are of  Verb, Noun, Pronouns, Adjectives, Adverbs, Adpositions, Conjunctions,Determiners, Cardinal Numbers, Particles, Other/ Foreign words, Punctuations
 
3. <b>Objectives </b>
  * First implement the vanilla Viterbi algorithm for assigning POS tags
  * Then use different methods (atleast 2) to tag unknown words.
  * Compute the accuracy for these two methods and compare with the vanilla Viterbi accuracy.
  * List down at least three cases from the sample test file (i.e. unknown word-tag pairs) which were incorrectly tagged by     the original Viterbi POS tagger and got corrected after  modifications.
 <hr />

# Steps
<hr />

1.   <b> Data Preparation and Exploring the Penn Treebank dataset </b>  

  
2.  <b> Build the vanilla Viterbi based POS tagger </b> 

      
3. <b> Modify Viterbi Heuristic Algorithm to consider only transition probability for UNKNOWN words </b> 


4.  <b> Modify Viterbi Heuristic Algorithm to tag UNKNOWN words using rule-based approach  </b> 


5.   <b> Evaluate each model on validation data </b>


5.   <b> Compare accuracy of modified models with vanilla Viterbi </b> 


6.  <b> Print word tags corrected by modified models </b> 


7. <b> Tag the words using these new models in the sample file and compare the results. </b> 

<hr />


## Data Preparation and EDA

In [188]:
#Importing libraries
import nltk
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import random
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import pprint, time
from nltk.tokenize import word_tokenize

from collections import Counter

In [189]:
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

In [190]:
# first few tagged sentences
print(nltk_data[:5])

[[('Pierre', 'NOUN'), ('Vinken', 'NOUN'), (',', '.'), ('61', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), (',', '.'), ('will', 'VERB'), ('join', 'VERB'), ('the', 'DET'), ('board', 'NOUN'), ('as', 'ADP'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('Nov.', 'NOUN'), ('29', 'NUM'), ('.', '.')], [('Mr.', 'NOUN'), ('Vinken', 'NOUN'), ('is', 'VERB'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Elsevier', 'NOUN'), ('N.V.', 'NOUN'), (',', '.'), ('the', 'DET'), ('Dutch', 'NOUN'), ('publishing', 'VERB'), ('group', 'NOUN'), ('.', '.')], [('Rudolph', 'NOUN'), ('Agnew', 'NOUN'), (',', '.'), ('55', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), ('and', 'CONJ'), ('former', 'ADJ'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Consolidated', 'NOUN'), ('Gold', 'NOUN'), ('Fields', 'NOUN'), ('PLC', 'NOUN'), (',', '.'), ('was', 'VERB'), ('named', 'VERB'), ('*-1', 'X'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('of', 'ADP'), ('this', 'DET'), ('British', 'ADJ'), ('industrial', 'ADJ'), ('

In [191]:
# Splitting into train and test
random.seed(1234)
# validation set is 5% of the total dataset
train_set,vald_set = train_test_split(nltk_data,random_state = 42,test_size=0.05)

print(len(train_set))
print(len(vald_set))

3718
196


In [192]:
train_tagged_words = [tup for sent in train_set for tup in sent]
len(train_tagged_words)


95589

In [193]:
# tokens 
tokens = [pair[0] for pair in train_tagged_words]
tokens[:10]

['Bank', 'of', 'New', 'England', "'s", 'shares', 'are', 'traded', '*-1', 'on']

In [194]:
# vocabulary
V = set(tokens)
print(len(V))

12109


In [195]:
# number of tags
T = set([pair[1] for pair in train_tagged_words])
len(T)
list(T)

['ADP',
 'X',
 'DET',
 'NUM',
 'VERB',
 'PRT',
 'ADJ',
 'CONJ',
 'PRON',
 'ADV',
 'NOUN',
 '.']

In [196]:
#We notice that there are many words that are assigned to .
dotpair = [pair for pair in train_tagged_words if pair[1]=='.']
Counter(dotpair).most_common(10)

[((',', '.'), 4632),
 (('.', '.'), 3638),
 (('``', '.'), 668),
 (('$', '.'), 667),
 (("''", '.'), 650),
 (('--', '.'), 214),
 ((';', '.'), 171),
 ((':', '.'), 135),
 (('-RRB-', '.'), 106),
 (('-LRB-', '.'), 103)]

## Vanilla Viterbi based POS tagger

### Emission Probabilities

In [197]:
# computing P(w/t) and storing in T x V matrix
t = len(T)
v = len(V)
w_given_t = np.zeros((t, v))

In [198]:
# compute word given tag: Emission Probability
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    count_w_given_tag = len(w_given_tag_list)
    
    return (count_w_given_tag, count_tag)

### Transition Probabilities

In [199]:
# compute tag given tag: tag2(t2) given tag1 (t1), i.e. Transition Probability

def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [200]:
# creating t x t transition matrix of tags
# each column is t2, each row is t1
# thus M(i, j) represents P(tj given ti)

tags_matrix = np.zeros((len(T), len(T)), dtype='float32')
for i, t1 in enumerate(list(T)):
    for j, t2 in enumerate(list(T)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]

In [201]:
# convert the matrix to a df for better readability
tags_df = pd.DataFrame(tags_matrix, columns = list(T), index=list(T))
tags_df

,ADP,X,DET,NUM,VERB,PRT,ADJ,CONJ,PRON,ADV,NOUN,.
ADP,0.017228,0.034029,0.326378,0.062921,0.008240,0.001498,0.105297,0.000856,0.069128,0.013162,0.321776,0.039486
X,0.142925,0.076482,0.055131,0.002709,0.203633,0.185787,0.016571,0.010357,0.056087,0.025175,0.061345,0.163799
DET,0.009054,0.045509,0.005311,0.021970,0.038387,0.000241,0.204973,0.000483,0.003742,0.012313,0.640029,0.017986
NUM,0.034790,0.206661,0.003271,0.184062,0.017544,0.027951,0.034196,0.013381,0.001487,0.002974,0.355338,0.118347
VERB,0.090493,0.218005,0.133101,0.022817,0.169189,0.031121,0.064649,0.005588,0.036244,0.082577,0.110904,0.035312
PRT,0.021576,0.013403,0.100360,0.058516,0.402746,0.001635,0.086303,0.002288,0.018960,0.010134,0.242563,0.041517
ADJ,0.078986,0.021392,0.005101,0.020405,0.012342,0.010861,0.066645,0.016949,0.000658,0.004608,0.696725,0.065328
CONJ,0.054435,0.007977,0.121539,0.042234,0.153918,0.004693,0.116847,0.000469,0.058658,0.055842,0.349132,0.034256
PRON,0.022918,0.092819,0.009549,0.007257,0.480901,0.012223,0.074866,0.004966,0.008021,0.033995,0.211230,0.041253
ADV,0.119601,0.023588,0.067110,0.030565,0.344518,0.013621,0.130233,0.006312,0.015615,0.081063,0.030897,0.136877


In [202]:
tags_df.loc['.', :]

ADP     0.091114
X       0.026623
DET     0.173502
NUM     0.080500
VERB    0.088505
PRT     0.002339
ADJ     0.044972
CONJ    0.057924
PRON    0.065389
ADV     0.052078
NOUN    0.223152
.       0.093812
Name: ., dtype: float32

In [203]:
len(train_tagged_words)

95589

In [204]:
# Viterbi Heuristic
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [205]:
def measure_accuracy(predicted, real):
    check = [i for i, j in zip(predicted, real) if i == j] 
    accuracy = len(check)/len(predicted)
    return accuracy

In [206]:
def find_incorrect_tags(predicted, real):
    incorrect_tags = [[real[i-1],j] for i, j in enumerate(zip(predicted, real)) if j[0]!=j[1]]
    return incorrect_tags

### Evaluating on validation set

In [207]:
# Running Viterbi algorithm on validation dataset


# list of tagged words
test_run_base = [tup for sent in vald_set for tup in sent]

# list of untagged words
test_tagged_words = [tup[0] for sent in vald_set for tup in sent]


In [208]:
# tagging the test sentences
start = time.time()
tagged_seq = Viterbi(test_tagged_words)
end = time.time()
difference = end-start
print("Time taken in seconds: ", difference)


Time taken in seconds:  1941.3890435695648


In [209]:
acc = measure_accuracy(tagged_seq,test_run_base)
print("Accuracy with original Viterbi \t\t{:0.2f} %".format(acc*100))


Accuracy with original Viterbi 		91.47 %


In [210]:
find_incorrect_tags(tagged_seq, test_run_base)

[[('expanded', 'VERB'), (('trade', 'VERB'), ('trade', 'NOUN'))],
 [('the', 'DET'), (('Overseas', 'ADP'), ('Overseas', 'NOUN'))],
 [('Overseas', 'NOUN'), (('Private', 'ADJ'), ('Private', 'NOUN'))],
 [('settled', 'VERB'), (('pre-1917', 'ADP'), ('pre-1917', 'ADJ'))],
 [('``', '.'), (('Unemployment', 'ADP'), ('Unemployment', 'NOUN'))],
 [('the', 'DET'), (('purchasing', 'NOUN'), ('purchasing', 'VERB'))],
 [('weekly', 'ADJ'), (('paycheck', 'ADP'), ('paycheck', 'NOUN'))],
 [('paycheck', 'NOUN'), (('reasonably', 'ADP'), ('reasonably', 'ADV'))],
 [(',', '.'), (('though', 'ADP'), ('though', 'ADV'))],
 [('such', 'ADJ'), (('close', 'NOUN'), ('close', 'ADJ'))],
 [('acquiring', 'VERB'), (('more', 'ADV'), ('more', 'ADJ'))],
 [('*-1', 'X'), (('Funded', 'ADP'), ('Funded', 'VERB'))],
 [('from', 'ADP'), (('Tokio', 'ADP'), ('Tokio', 'NOUN'))],
 [('medical', 'ADJ'), (('protocols', 'ADP'), ('protocols', 'NOUN'))],
 [('on', 'ADP'), (('preventative', 'ADP'), ('preventative', 'ADJ'))],
 [('it', 'PRON'), (('exi

In [211]:
tagged_seq

[('For', 'ADP'),
 ('the', 'DET'),
 ('Agency', 'NOUN'),
 ('for', 'ADP'),
 ('International', 'NOUN'),
 ('Development', 'NOUN'),
 (',', '.'),
 ('appropriators', 'NOUN'),
 ('approved', 'VERB'),
 ('$', '.'),
 ('200', 'NUM'),
 ('million', 'NUM'),
 ('*U*', 'X'),
 ('in', 'ADP'),
 ('secondary', 'ADJ'),
 ('loan', 'NOUN'),
 ('guarantees', 'NOUN'),
 ('under', 'ADP'),
 ('an', 'DET'),
 ('expanded', 'VERB'),
 ('trade', 'VERB'),
 ('credit', 'NOUN'),
 ('insurance', 'NOUN'),
 ('program', 'NOUN'),
 (',', '.'),
 ('and', 'CONJ'),
 ('total', 'ADJ'),
 ('loan', 'NOUN'),
 ('guarantees', 'NOUN'),
 ('for', 'ADP'),
 ('the', 'DET'),
 ('Overseas', 'ADP'),
 ('Private', 'ADJ'),
 ('Investment', 'NOUN'),
 ('Corp.', 'NOUN'),
 ('are', 'VERB'),
 ('increased', 'VERB'),
 ('*-3', 'X'),
 ('by', 'ADP'),
 ('$', '.'),
 ('40', 'NUM'),
 ('million', 'NUM'),
 ('*U*', 'X'),
 ('over', 'ADP'),
 ('fiscal', 'ADJ'),
 ('1989', 'NUM'),
 ('as', 'ADP'),
 ('part', 'NOUN'),
 ('of', 'ADP'),
 ('the', 'DET'),
 ('same', 'ADJ'),
 ('Poland', 'NOUN'),

## Model 1: Consider only transition probability for unknown words
#### Viterbi Algorithm with modification to Emission and Transition Probability

- If the emission probability for a word is 0 i.e new word, let the transition probability alone decide the new tag. 

In [212]:
#Viterbi Heuristic
def Viterbi_only_transition_prob(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        trans=[]
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p
            p.append(state_probability)
            trans.append(transition_p)
    
        pmax = max(p)
        # getting state for which probability is maximum
        if pmax == 0: #Emission probability is 0 and hence it is a new word.
            pmax = max(trans) # In such case find the maximum transition probability.
            state_max = T[trans.index(pmax)] # Let this transition prob decide the new tag.
        else:
            state_max = T[p.index(pmax)] # If emission prob !=0 then let the state_probability decide the tag as usual. 
        state.append(state_max)
    return list(zip(words, state))


In [213]:
start = time.time()
m1_tagged_seq = Viterbi_only_transition_prob(test_tagged_words)
end = time.time()
difference = end-start
print("Time taken in seconds: ",difference)


Time taken in seconds:  1916.6774547100067


In [214]:
acc = measure_accuracy(m1_tagged_seq,test_run_base)
print("Accuracy with model1 Viterbi \t\t{:0.2f} %".format(acc*100))

Accuracy with model1 Viterbi 		93.89 %


In [215]:
find_incorrect_tags(m1_tagged_seq, test_run_base)

[[('expanded', 'VERB'), (('trade', 'VERB'), ('trade', 'NOUN'))],
 [('Overseas', 'NOUN'), (('Private', 'ADJ'), ('Private', 'NOUN'))],
 [('settled', 'VERB'), (('pre-1917', 'X'), ('pre-1917', 'ADJ'))],
 [('the', 'DET'), (('purchasing', 'NOUN'), ('purchasing', 'VERB'))],
 [('paycheck', 'NOUN'), (('reasonably', 'NOUN'), ('reasonably', 'ADV'))],
 [(',', '.'), (('though', 'ADP'), ('though', 'ADV'))],
 [('such', 'ADJ'), (('close', 'NOUN'), ('close', 'ADJ'))],
 [('acquiring', 'VERB'), (('more', 'ADV'), ('more', 'ADJ'))],
 [('from', 'ADP'), (('Tokio', 'DET'), ('Tokio', 'NOUN'))],
 [('on', 'ADP'), (('preventative', 'DET'), ('preventative', 'ADJ'))],
 [('$', '.'), (('20.5', 'NOUN'), ('20.5', 'NUM'))],
 [('become', 'VERB'), (('polarized', 'X'), ('polarized', 'VERB'))],
 [(',', '.'), (('so', 'ADV'), ('so', 'ADP'))],
 [('a', 'DET'), (('middle', 'NOUN'), ('middle', 'ADJ'))],
 [('.', '.'), (('Though', 'NOUN'), ('Though', 'ADP'))],
 [('totaled', 'VERB'), (('154.2', 'X'), ('154.2', 'NUM'))],
 [('Since', 

### Compare the tagging accuracies of the model1  with vanilla Viterbi algorithm
- Compared to the 91.47% accuracy of vanilla Viterbi algorithm, the Viterbi considering only transition probability for unknown words gives an accuracy of almost 93.89%. 

### Words tagged correctly by model1 which were incorrectly tagged by vanilla Viterbi

Some of the cases where the modified algorithm worked better are :
    - ('Unemployment', 'NUM') was rightly tagged as ('Unemployment', 'NOUN')
    - ('Overseas', 'NUM') was rightly tagged as ('Overseas', 'NOUN')
    - ('paycheck', 'NUM') was rightly tagged as ('paycheck', 'NOUN')
    - ('Funded', 'NUM') was rightly tagged as ('Funded', 'VERB')
    - ('protocols', 'NUM') was rightly tagged as ('protocols', 'NOUN')

## Model 2: Tag unknown words using rule based algorithm

In [216]:
def rule_based(words, state):
    
    # specify patterns for tagging
    # example from the NLTK book
    patterns = [
        (r'.*ing$', 'VERB'),              # gerund
        (r'.*ed$', 'VERB'),               # past tense
        (r'.*es$', 'VERB'),               # 3rd singular present
        (r'.*ould$', 'MD'),              # modals
        (r'.*\'s$', 'NN$'),              # possessive nouns
        (r'.*s$', 'NNS'),                # plural nouns
        (r'.*\*.*', 'X'),                # plural nouns 
        (r'([0-9])+.*$', 'NUM'), # cardinal numbers
        (r'.*', 'NOUN')                    # nouns
    ]
    
    regexp_tagger = nltk.RegexpTagger(patterns)
    
    for i,j in enumerate(state):
        #Rule-based tagging done only for new 'unknown' words.
        if j == "unknown":
            state[i] = regexp_tagger.tag([words[i]])[0][1]
    return words,state

In [217]:
# Modified Viterbi Heuristic with Rule Based
def Viterbi_rule_based(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else: 
                #If previous tag is 'unknown' make transition_p = 1, so that emission_p alone decides the tag. 
                if (state[-1] == 'unknown'):
                    transition_p = 1
                else:             
                    transition_p = tags_df.loc[state[-1], tag]
                
           
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p
            p.append(state_probability)   
        pmax = max(p)

        if pmax == 0:
            #if pmax = 0, means the the word is a new word, in such case mark the word tag as "unknown".
            state_max = "unknown"
        else:
            state_max = T[p.index(pmax)]
        state.append(state_max)
    
    word, state = rule_based(words, state) #Rule-based function called for all 'unknown' new words.
    return list(zip(words, state))

In [218]:
# Tagging the test sentences

start = time.time()
m2_tagged_seq = Viterbi_rule_based(test_tagged_words)
end = time.time()
difference = end-start
print("Time taken in seconds: ", difference)


Time taken in seconds:  1959.3257241249084


In [219]:
acc = measure_accuracy(m2_tagged_seq,test_run_base)
print("Accuracy with model2 Viterbi \t\t{:0.2f} %".format(acc*100))

Accuracy with model2 Viterbi 		95.03 %


In [220]:
find_incorrect_tags(m2_tagged_seq, test_run_base)

[[('expanded', 'VERB'), (('trade', 'VERB'), ('trade', 'NOUN'))],
 [('the', 'DET'), (('Overseas', 'NNS'), ('Overseas', 'NOUN'))],
 [('Overseas', 'NOUN'), (('Private', 'ADJ'), ('Private', 'NOUN'))],
 [('settled', 'VERB'), (('pre-1917', 'NOUN'), ('pre-1917', 'ADJ'))],
 [('the', 'DET'), (('purchasing', 'NOUN'), ('purchasing', 'VERB'))],
 [('paycheck', 'NOUN'), (('reasonably', 'NOUN'), ('reasonably', 'ADV'))],
 [(',', '.'), (('though', 'ADP'), ('though', 'ADV'))],
 [('such', 'ADJ'), (('close', 'NOUN'), ('close', 'ADJ'))],
 [('acquiring', 'VERB'), (('more', 'ADV'), ('more', 'ADJ'))],
 [('medical', 'ADJ'), (('protocols', 'NNS'), ('protocols', 'NOUN'))],
 [('on', 'ADP'), (('preventative', 'NOUN'), ('preventative', 'ADJ'))],
 [(',', '.'), (('so', 'ADV'), ('so', 'ADP'))],
 [('*ICH*-2', 'X'), (('exists', 'NNS'), ('exists', 'VERB'))],
 [('a', 'DET'), (('middle', 'NOUN'), ('middle', 'ADJ'))],
 [('.', '.'), (('Though', 'NOUN'), ('Though', 'ADP'))],
 [('prospective', 'ADJ'), (('acquirers', 'NNS'), ('

### Compare the tagging accuracies of the model2  with vanilla Viterbi algorithm
- Compared to the 91.47% accuracy of vanilla Viterbi algorithm, the rule based Viterbi gives an accuracy of almost 95.03%. 

### Words tagged correctly by model2 which were incorrectly tagged by vanilla Viterbi

Some of the cases where the modified algorithm worked better are :
    - ('*T*-253', 'NUM') was now rightly tagged as ('*T*-253', 'X'))
    - ('*T*-222', 'NUM') was now rightly tagged as ('*T*-222', 'X'))
    - ('existed', 'NUM') was now rightly tagged as ('existed', 'VERB'))
    - ('noticed', 'NUM') was now rightly tagged as ('noticed', 'VERB'))
    - ('schoolchildren', 'NUM') was now rightly tagged as ('schoolchildren', 'NOUN')

## Model 3: Tag unknown words using most commonly used tag

In [221]:
# find the most commonly used tag in training dataset
#Extract words and tags from training data set
words = [pair[0].lower() for pair in train_tagged_words]
tags = [pair[1] for pair in train_tagged_words]
tags_counter = Counter(tags)

#Finding the most common tag
print("Most common tags in the given training data set : \n {0}".format(tags_counter.most_common(5)))


Most common tags in the given training data set : 
 [('NOUN', 27423), ('VERB', 12885), ('.', 11118), ('ADP', 9345), ('DET', 8284)]


<b> We notice that "NOUN" is the most common tag in the training data set. 

In [222]:
def most_common_tag(words, state):
    
    # specify patterns for tagging
    # example from the NLTK book
    patterns = [
        (r'.*ing$', 'VERB'),              # gerund
        (r'.*ed$', 'VERB'),               # past tense
        (r'.*es$', 'VERB'),               # 3rd singular present
        (r'.*ould$', 'MD'),              # modals
        (r'.*\'s$', 'NN$'),              # possessive nouns
        (r'.*s$', 'NNS'),                # plural nouns
        (r'.*\*.*', 'X'),                # plural nouns 
        (r'([0-9])+.*$', 'NUM'), # cardinal numbers
        (r'.*', 'NOUN')                    # nouns
    ]
    
    regexp_tagger = nltk.RegexpTagger(patterns)
    
    for i,j in enumerate(state):
        #Rule-based tagging done only for new 'unknown' words.
        if j == "unknown":
            state[i] = 'NOUN'
    return words,state

In [223]:
# Modified Viterbi Heuristic with Rule Based
def Viterbi_most_common_tag(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else: 
                #If previous tag is 'unknown' make transition_p = 1, so that emission_p alone decides the tag. 
                if (state[-1] == 'unknown'):
                    transition_p = 1
                else:             
                    transition_p = tags_df.loc[state[-1], tag]
                
           
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p
            p.append(state_probability)   
        pmax = max(p)

        if pmax == 0:
            #if pmax = 0, means the the word is a new word, in such case mark the word tag as "unknown".
            state_max = "unknown"
        else:
            state_max = T[p.index(pmax)]
        state.append(state_max)
    
    word, state = most_common_tag(words, state) #Rule-based function called for all 'unknown' new words.
    return list(zip(words, state))

In [224]:
# Tagging the test sentences

start = time.time()
m3_tagged_seq = Viterbi_most_common_tag(test_tagged_words)
end = time.time()
difference = end-start
print("Time taken in seconds: ", difference)


Time taken in seconds:  1921.6285433769226


In [225]:
acc = measure_accuracy(m3_tagged_seq,test_run_base)
print("Accuracy with model3 Viterbi \t\t{:0.2f} %".format(acc*100))

Accuracy with model3 Viterbi 		93.95 %


### Compare the tagging accuracies of the model3  with vanilla Viterbi algorithm
- Compared to the 91.47% accuracy of vanilla Viterbi algorithm, the rule based Viterbi gives an accuracy of almost 94% 

# Evaluating models on sentences in sample test file

### Run vanilla Viterbi on the sentences in sample file

In [226]:
nltk.download('punkt')
with open('sample_text.txt', 'r') as myfile:
    test_sentence = myfile.read()
    sentence_words = word_tokenize(test_sentence)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bindus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [227]:
nltk.download('averaged_perceptron_tagger')
sampleFile_tags = nltk.pos_tag(sentence_words, tagset='universal')
sampleFile_tags

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\bindus\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


[('Android', 'NOUN'),
 ('is', 'VERB'),
 ('a', 'DET'),
 ('mobile', 'ADJ'),
 ('operating', 'NOUN'),
 ('system', 'NOUN'),
 ('developed', 'VERB'),
 ('by', 'ADP'),
 ('Google', 'NOUN'),
 ('.', '.'),
 ('Android', 'NOUN'),
 ('has', 'VERB'),
 ('been', 'VERB'),
 ('the', 'DET'),
 ('best-selling', 'ADJ'),
 ('OS', 'NOUN'),
 ('worldwide', 'NOUN'),
 ('on', 'ADP'),
 ('smartphones', 'NOUN'),
 ('since', 'ADP'),
 ('2011', 'NUM'),
 ('and', 'CONJ'),
 ('on', 'ADP'),
 ('tablets', 'NOUN'),
 ('since', 'ADP'),
 ('2013', 'NUM'),
 ('.', '.'),
 ('Google', 'NOUN'),
 ('and', 'CONJ'),
 ('Twitter', 'NOUN'),
 ('made', 'VERB'),
 ('a', 'DET'),
 ('deal', 'NOUN'),
 ('in', 'ADP'),
 ('2015', 'NUM'),
 ('that', 'DET'),
 ('gave', 'VERB'),
 ('Google', 'NOUN'),
 ('access', 'NOUN'),
 ('to', 'PRT'),
 ('Twitter', 'NOUN'),
 ("'s", 'PRT'),
 ('firehose', 'NOUN'),
 ('.', '.'),
 ('Twitter', 'NOUN'),
 ('is', 'VERB'),
 ('an', 'DET'),
 ('online', 'ADJ'),
 ('news', 'NOUN'),
 ('and', 'CONJ'),
 ('social', 'ADJ'),
 ('networking', 'NOUN'),
 ('se

In [228]:
# list of words
sampleFile_words = [tup[0] for tup in sampleFile_tags]
#sampleFile_words

In [229]:
start = time.time()
sampleFile_tags_plain = Viterbi(sampleFile_words)
end = time.time()
difference = end-start
print("Time taken in seconds: ", difference)

Time taken in seconds:  67.31965398788452


In [230]:
acc = measure_accuracy(sampleFile_tags_plain, sampleFile_tags)

print("Accuracy with plain Viterbi \t\t{:0.2f} %".format(acc*100))

Accuracy with plain Viterbi 		76.24 %


### Run modified Viterbi to consider only transitional probabilities on the sentences in sample file

In [231]:
start = time.time()
sampleFile_tags_m2 = Viterbi_only_transition_prob(sampleFile_words)
end = time.time()
difference = end-start
print("Time taken in seconds: ", difference)
sampleFile_tags_m2

Time taken in seconds:  78.9206337928772


[('Android', 'NOUN'),
 ('is', 'VERB'),
 ('a', 'DET'),
 ('mobile', 'ADJ'),
 ('operating', 'NOUN'),
 ('system', 'NOUN'),
 ('developed', 'VERB'),
 ('by', 'ADP'),
 ('Google', 'DET'),
 ('.', '.'),
 ('Android', 'NOUN'),
 ('has', 'VERB'),
 ('been', 'VERB'),
 ('the', 'DET'),
 ('best-selling', 'ADJ'),
 ('OS', 'NOUN'),
 ('worldwide', 'NOUN'),
 ('on', 'ADP'),
 ('smartphones', 'DET'),
 ('since', 'ADP'),
 ('2011', 'DET'),
 ('and', 'CONJ'),
 ('on', 'ADP'),
 ('tablets', 'NOUN'),
 ('since', 'ADP'),
 ('2013', 'DET'),
 ('.', '.'),
 ('Google', 'NOUN'),
 ('and', 'CONJ'),
 ('Twitter', 'NOUN'),
 ('made', 'VERB'),
 ('a', 'DET'),
 ('deal', 'NOUN'),
 ('in', 'ADP'),
 ('2015', 'DET'),
 ('that', 'ADP'),
 ('gave', 'VERB'),
 ('Google', 'X'),
 ('access', 'NOUN'),
 ('to', 'PRT'),
 ('Twitter', 'VERB'),
 ("'s", 'PRT'),
 ('firehose', 'VERB'),
 ('.', '.'),
 ('Twitter', 'NOUN'),
 ('is', 'VERB'),
 ('an', 'DET'),
 ('online', 'NOUN'),
 ('news', 'NOUN'),
 ('and', 'CONJ'),
 ('social', 'ADJ'),
 ('networking', 'NOUN'),
 ('servic

In [232]:
acc = measure_accuracy(sampleFile_tags_m2, sampleFile_tags)
print("Accuracy with modified Viterbi with transitional probabilities \t\t{:0.2f} %".format(acc*100))

Accuracy with modified Viterbi with transitional probabilities 		86.74 %


### Words tagged correctly by model1 which were incorrectly tagged by vanilla Viterbi
Some of the cases where the modified algorithm worked better are :

- ('experience', 'VERB') is now rightly tagged as ('experience', 'NOUN')
- ('Show', 'VERB') is now rightly tagged as ('Show', 'NOUN')
- ('contested', 'NOUN') is now rightly tagged as ('contested', 'VERB')
- ('that', 'DET') is now rightly tagged as ('that', 'ADP')

### Run modified rule based Viterbi on the sentences in sample file

In [233]:
start = time.time()
sampleFile_tags_m3 = Viterbi_rule_based(sampleFile_words)
end = time.time()
difference = end-start
print("Time taken in seconds: ", difference)
sampleFile_tags_m3

Time taken in seconds:  67.73312711715698


[('Android', 'NOUN'),
 ('is', 'VERB'),
 ('a', 'DET'),
 ('mobile', 'ADJ'),
 ('operating', 'NOUN'),
 ('system', 'NOUN'),
 ('developed', 'VERB'),
 ('by', 'ADP'),
 ('Google', 'NOUN'),
 ('.', '.'),
 ('Android', 'NOUN'),
 ('has', 'VERB'),
 ('been', 'VERB'),
 ('the', 'DET'),
 ('best-selling', 'ADJ'),
 ('OS', 'NOUN'),
 ('worldwide', 'NOUN'),
 ('on', 'ADP'),
 ('smartphones', 'VERB'),
 ('since', 'ADP'),
 ('2011', 'NUM'),
 ('and', 'CONJ'),
 ('on', 'ADP'),
 ('tablets', 'NOUN'),
 ('since', 'ADP'),
 ('2013', 'NUM'),
 ('.', '.'),
 ('Google', 'NOUN'),
 ('and', 'CONJ'),
 ('Twitter', 'NOUN'),
 ('made', 'VERB'),
 ('a', 'DET'),
 ('deal', 'NOUN'),
 ('in', 'ADP'),
 ('2015', 'NUM'),
 ('that', 'ADP'),
 ('gave', 'VERB'),
 ('Google', 'NOUN'),
 ('access', 'NOUN'),
 ('to', 'PRT'),
 ('Twitter', 'NOUN'),
 ("'s", 'PRT'),
 ('firehose', 'NOUN'),
 ('.', '.'),
 ('Twitter', 'NOUN'),
 ('is', 'VERB'),
 ('an', 'DET'),
 ('online', 'NOUN'),
 ('news', 'NOUN'),
 ('and', 'CONJ'),
 ('social', 'ADJ'),
 ('networking', 'NOUN'),
 ('s

In [234]:
acc = measure_accuracy(sampleFile_tags_m3, sampleFile_tags)
print("Accuracy with modified Viterbi which is rule based \t\t{:0.2f} %".format(acc*100))

Accuracy with modified Viterbi which is rule based 		92.27 %


### Words tagged correctly by model2 which were incorrectly tagged by vanilla Viterbi
Some of the cases where the modified algorithm worked better are :

- ('online', 'ADJ') is now rightly tagged as ('online', 'NOUN')
- ('arriving', 'NOUN') is now rightly tagged as ('arriving', 'VERB')
- ('like', 'VERB') is now rightly tagged as ('like', 'ADP')
- ('trips', 'NOUN') is now rightly tagged as ('trips', 'NNS')

### Run modified Viterbi that tags unknown words with most common tag on the sentences in sample file

In [235]:
start = time.time()
sampleFile_tags_m4 = Viterbi_most_common_tag(sampleFile_words)
end = time.time()
difference = end-start
print("Time taken in seconds: ", difference)
sampleFile_tags_m4

Time taken in seconds:  68.8569540977478


[('Android', 'NOUN'),
 ('is', 'VERB'),
 ('a', 'DET'),
 ('mobile', 'ADJ'),
 ('operating', 'NOUN'),
 ('system', 'NOUN'),
 ('developed', 'VERB'),
 ('by', 'ADP'),
 ('Google', 'NOUN'),
 ('.', '.'),
 ('Android', 'NOUN'),
 ('has', 'VERB'),
 ('been', 'VERB'),
 ('the', 'DET'),
 ('best-selling', 'ADJ'),
 ('OS', 'NOUN'),
 ('worldwide', 'NOUN'),
 ('on', 'ADP'),
 ('smartphones', 'NOUN'),
 ('since', 'ADP'),
 ('2011', 'NOUN'),
 ('and', 'CONJ'),
 ('on', 'ADP'),
 ('tablets', 'NOUN'),
 ('since', 'ADP'),
 ('2013', 'NOUN'),
 ('.', '.'),
 ('Google', 'NOUN'),
 ('and', 'CONJ'),
 ('Twitter', 'NOUN'),
 ('made', 'VERB'),
 ('a', 'DET'),
 ('deal', 'NOUN'),
 ('in', 'ADP'),
 ('2015', 'NOUN'),
 ('that', 'ADP'),
 ('gave', 'VERB'),
 ('Google', 'NOUN'),
 ('access', 'NOUN'),
 ('to', 'PRT'),
 ('Twitter', 'NOUN'),
 ("'s", 'PRT'),
 ('firehose', 'NOUN'),
 ('.', '.'),
 ('Twitter', 'NOUN'),
 ('is', 'VERB'),
 ('an', 'DET'),
 ('online', 'NOUN'),
 ('news', 'NOUN'),
 ('and', 'CONJ'),
 ('social', 'ADJ'),
 ('networking', 'NOUN'),
 

In [237]:
acc = measure_accuracy(sampleFile_tags_m4, sampleFile_tags)
print("Accuracy with modified Viterbi where the most common tag is picked is \t\t{:0.2f} %".format(acc*100))

Accuracy with modified Viterbi where the most common tag is picked is 		91.16 %


### Words tagged correctly by model3  which were incorrectly tagged by vanilla Viterbi
Some of the cases where the modified algorithm worked better are:

- ('Show', 'VERB') is now correctly tagged as ('Show', 'NOUN')
- ('11th', 'NUM') is now rightly tagged as ('11th', 'ADJ')
- ('like', 'VERB') is now rightly tagged as ('like', 'ADP')
- ('about', 'ADV') is now rightly tagged as ('about', 'ADP')

# Conclusions

1. The rule based Viterbi models perform much better than vanilla Viterbi and the Viterbi model that considers only transitional   probabilities for unknown words.

2. There are still some incorrect tags presennt after each of the methods, which may not be gramatically correct in a given context. This may be mainly because of the limited number of tags (12) in the dataset.
